In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
import sys
import os 
sys.path.append("../student_marks")

In [3]:
repo_root = os.path.abspath("..")
gt_path = os.path.join(repo_root, "data/student_grades_ground_truth.xlsx")
preds_path = os.path.join(repo_root, "data/student_grades.xlsx")
student_grades_gt = pd.read_excel(gt_path)
student_grade_predictions = pd.read_excel(preds_path)

In [4]:
student_grades_gt["student_id"] = student_grades_gt["student_id"].ffill()
student_grades_gt["report"] = student_grades_gt["report"].ffill()
student_grades_gt["global_grade"] = student_grades_gt["global_grade"].ffill()
student_grades_gt

,student_id,skill,report,grade,weight,global_grade
0,student_1,academic_skills,academic_skills: The student has demonstrate...,10,0.3,10.0
1,student_1,socioemotional_skills,academic_skills: The student has demonstrate...,10,0.1,10.0
2,student_1,learning_and_thinking_skills,academic_skills: The student has demonstrate...,10,0.2,10.0
3,student_1,communication_skills,academic_skills: The student has demonstrate...,10,0.2,10.0
4,student_1,guidance_skills,academic_skills: The student has demonstrate...,10,0.2,10.0
5,student_2,academic_skills,academic_skills: The student shows interest ...,6,0.3,6.2
6,student_2,socioemotional_skills,academic_skills: The student shows interest ...,8,0.1,6.2
7,student_2,learning_and_thinking_skills,academic_skills: The student shows interest ...,6,0.2,6.2
8,student_2,communication_skills,academic_skills: The student shows interest ...,6,0.2,6.2
9,student_2,guidance_skills,academic_skills: The student shows interest ...,6,0.2,6.2


In [5]:
student_grade_predictions

,student_id,skill,grade,explanation,relevant_chunks,global_grade
0,student_1,academic_skills,8,The student received the grade 8 for the compe...,['academic_skills: The student has demonstrate...,8.0
1,student_1,socioemotional_skills,8,The student received the grade 8 for the compe...,['skills: The student shows great autonomy in ...,8.0
2,student_1,learning_and_thinking_skills,8,The student received the grade 8 for the compe...,['academic_skills: The student has demonstrate...,8.0
3,student_1,communication_skills,8,The student communicates clearly and effective...,['es and time.\ncommunication_skills: The stud...,8.0
4,student_1,guidance_skills,8,The student received the grade 8 for the compe...,"[', perseverance, and academic self-esteem.', ...",8.0
5,student_2,academic_skills,6,The student received the grade 6 for the compe...,['academic_skills: The student shows interest ...,6.8
6,student_2,socioemotional_skills,6,"He shows interest in learning, although his mo...","['skills. He shows interest in learning, altho...",6.8
7,student_2,learning_and_thinking_skills,8,The student received the grade 8 for the compe...,['nagement skills.\nlearning_and_thinking_skil...,6.8
8,student_2,communication_skills,8,The student received the grade 8 for the compe...,['lan and organize activities more efficiently...,6.8
9,student_2,guidance_skills,6,The student is adapting to the school environm...,"['skills. He shows interest in learning, altho...",6.8


In [6]:
student_grades_gt_preds = student_grades_gt.merge(student_grade_predictions, on=["student_id", "skill"], suffixes=('_gt', '_pred'))
student_grades_gt_preds

,student_id,skill,report,grade_gt,weight,global_grade_gt,grade_pred,explanation,relevant_chunks,global_grade_pred
0,student_1,academic_skills,academic_skills: The student has demonstrate...,10,0.3,10.0,8,The student received the grade 8 for the compe...,['academic_skills: The student has demonstrate...,8.0
1,student_1,socioemotional_skills,academic_skills: The student has demonstrate...,10,0.1,10.0,8,The student received the grade 8 for the compe...,['skills: The student shows great autonomy in ...,8.0
2,student_1,learning_and_thinking_skills,academic_skills: The student has demonstrate...,10,0.2,10.0,8,The student received the grade 8 for the compe...,['academic_skills: The student has demonstrate...,8.0
3,student_1,communication_skills,academic_skills: The student has demonstrate...,10,0.2,10.0,8,The student communicates clearly and effective...,['es and time.\ncommunication_skills: The stud...,8.0
4,student_1,guidance_skills,academic_skills: The student has demonstrate...,10,0.2,10.0,8,The student received the grade 8 for the compe...,"[', perseverance, and academic self-esteem.', ...",8.0
5,student_2,academic_skills,academic_skills: The student shows interest ...,6,0.3,6.2,6,The student received the grade 6 for the compe...,['academic_skills: The student shows interest ...,6.8
6,student_2,socioemotional_skills,academic_skills: The student shows interest ...,8,0.1,6.2,6,"He shows interest in learning, although his mo...","['skills. He shows interest in learning, altho...",6.8
7,student_2,learning_and_thinking_skills,academic_skills: The student shows interest ...,6,0.2,6.2,8,The student received the grade 8 for the compe...,['nagement skills.\nlearning_and_thinking_skil...,6.8
8,student_2,communication_skills,academic_skills: The student shows interest ...,6,0.2,6.2,8,The student received the grade 8 for the compe...,['lan and organize activities more efficiently...,6.8
9,student_2,guidance_skills,academic_skills: The student shows interest ...,6,0.2,6.2,6,The student is adapting to the school environm...,"['skills. He shows interest in learning, altho...",6.8


In [7]:
student_grades_gt_preds["Error skill"] = student_grades_gt_preds["grade_gt"] - student_grades_gt_preds["grade_pred"]
student_grades_gt_preds["Error global grade"] = student_grades_gt_preds["global_grade_gt"] - student_grades_gt_preds["global_grade_pred"]

In [8]:
student_grades_gt_preds

,student_id,skill,report,grade_gt,weight,global_grade_gt,grade_pred,explanation,relevant_chunks,global_grade_pred,Error skill,Error global grade
0,student_1,academic_skills,academic_skills: The student has demonstrate...,10,0.3,10.0,8,The student received the grade 8 for the compe...,['academic_skills: The student has demonstrate...,8.0,2,2.0
1,student_1,socioemotional_skills,academic_skills: The student has demonstrate...,10,0.1,10.0,8,The student received the grade 8 for the compe...,['skills: The student shows great autonomy in ...,8.0,2,2.0
2,student_1,learning_and_thinking_skills,academic_skills: The student has demonstrate...,10,0.2,10.0,8,The student received the grade 8 for the compe...,['academic_skills: The student has demonstrate...,8.0,2,2.0
3,student_1,communication_skills,academic_skills: The student has demonstrate...,10,0.2,10.0,8,The student communicates clearly and effective...,['es and time.\ncommunication_skills: The stud...,8.0,2,2.0
4,student_1,guidance_skills,academic_skills: The student has demonstrate...,10,0.2,10.0,8,The student received the grade 8 for the compe...,"[', perseverance, and academic self-esteem.', ...",8.0,2,2.0
5,student_2,academic_skills,academic_skills: The student shows interest ...,6,0.3,6.2,6,The student received the grade 6 for the compe...,['academic_skills: The student shows interest ...,6.8,0,-0.6
6,student_2,socioemotional_skills,academic_skills: The student shows interest ...,8,0.1,6.2,6,"He shows interest in learning, although his mo...","['skills. He shows interest in learning, altho...",6.8,2,-0.6
7,student_2,learning_and_thinking_skills,academic_skills: The student shows interest ...,6,0.2,6.2,8,The student received the grade 8 for the compe...,['nagement skills.\nlearning_and_thinking_skil...,6.8,-2,-0.6
8,student_2,communication_skills,academic_skills: The student shows interest ...,6,0.2,6.2,8,The student received the grade 8 for the compe...,['lan and organize activities more efficiently...,6.8,-2,-0.6
9,student_2,guidance_skills,academic_skills: The student shows interest ...,6,0.2,6.2,6,The student is adapting to the school environm...,"['skills. He shows interest in learning, altho...",6.8,0,-0.6


## Error per student

In [9]:
error_per_student = student_grades_gt_preds.groupby("student_id").first()[["global_grade_gt", "global_grade_pred", "Error global grade"]]
error_per_student

,global_grade_gt,global_grade_pred,Error global grade
student_id,,,
student_1,10.0,8.0,2.0
student_2,6.2,6.8,-0.6
student_3,4.0,5.4,-1.4
student_4,5.2,6.4,-1.2


In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=error_per_student.index,
    y=error_per_student["global_grade_gt"],
    mode="markers",
    name="Ground truth"
))
fig.add_trace(go.Scatter(
    x=error_per_student.index,
    y=error_per_student["global_grade_pred"],
    mode="markers",
    name="IA"
))

fig.update_layout(
    xaxis_title="Student",
    yaxis_title="Global grade",
    xaxis=dict(tickangle=45)
)

fig.show()


In [11]:
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=error_per_student.index,
    y=error_per_student["Error global grade"],
    mode="markers"
))

fig.add_hline(y=1.5, line_dash="dash", line_color="red")
fig.add_hline(y=-1.5, line_dash="dash", line_color="red")

fig.update_layout(
    xaxis_title="Student",
    yaxis_title="Error global grade (GT-IA)",
    xaxis=dict(tickangle=45)
)

fig.show()


## Error per skill

In [12]:
error_per_skill = student_grades_gt_preds.groupby("skill")["Error skill"].apply(lambda x: x.abs().mean())
error_per_skill

skill
academic_skills                 0.5
communication_skills            2.0
guidance_skills                 1.5
learning_and_thinking_skills    2.0
socioemotional_skills           1.5
Name: Error skill, dtype: float64

In [13]:
error_per_skill = pd.DataFrame(error_per_skill)

In [14]:
for skill in error_per_skill.index:
    error_per_skill.loc[skill, "Number students error other than 0"] = len(student_grades_gt_preds.loc[(student_grades_gt_preds["skill"]==skill)&(student_grades_gt_preds["Error global grade"]!=0)])
    error_per_skill.loc[skill, "Number students absoluto error greater than 1"] = len(student_grades_gt_preds.loc[(student_grades_gt_preds["skill"]==skill)&(abs(student_grades_gt_preds["Error global grade"])>1)])
    error_per_skill.loc[skill, "Number students absolute error greater than 2"] = len(student_grades_gt_preds.loc[(student_grades_gt_preds["skill"]==skill)&(abs(student_grades_gt_preds["Error global grade"])>2)])
    error_per_skill.loc[skill, "Number students absolute error equal to 10"] = len(student_grades_gt_preds.loc[(student_grades_gt_preds["skill"]==skill)&(abs(student_grades_gt_preds["Error global grade"])==10)])
    error_per_skill.loc[skill, "Number students IA less restrictive"] = len(student_grades_gt_preds.loc[(student_grades_gt_preds["skill"]==skill)&(student_grades_gt_preds["grade_gt"]>student_grades_gt_preds["grade_pred"])])
    error_per_skill.loc[skill, "Number students IA more or equal restrictive"] = len(student_grades_gt_preds.loc[(student_grades_gt_preds["skill"]==skill)&(student_grades_gt_preds["grade_gt"]<=student_grades_gt_preds["grade_pred"])])

In [15]:
error_per_skill

,Error skill,Number students error other than 0,Number students absoluto error greater than 1,Number students absolute error greater than 2,Number students absolute error equal to 10,Number students IA less restrictive,Number students IA more or equal restrictive
skill,,,,,,,
academic_skills,0.5,4.0,3.0,0.0,0.0,1.0,3.0
communication_skills,2.0,4.0,3.0,0.0,0.0,1.0,3.0
guidance_skills,1.5,4.0,3.0,0.0,0.0,1.0,3.0
learning_and_thinking_skills,2.0,4.0,3.0,0.0,0.0,1.0,3.0
socioemotional_skills,1.5,4.0,3.0,0.0,0.0,2.0,2.0


In [16]:
fig = go.Figure()


fig.add_trace(go.Scatter(
    x=error_per_skill.index,
    y=error_per_skill["Error skill"],
    mode="markers",
))

fig.add_hline(y=1.75, line_dash="dash", line_color="red")


fig.update_layout(
    xaxis_title="Skill",
    yaxis_title="Error skill (GT-IA)",
    xaxis=dict(tickangle=45)
)

fig.show()
